In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="Bank_Marketing_HyperDrive")

print('WORKSPACE DETAILS:', 'Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

WORKSPACE DETAILS:
Workspace name: quick-starts-ws-134051
Azure region: southcentralus
Subscription id: 5a4ab2ba-6c51-4805-8155-58759ad589d8
Resource group: aml-quickstarts-134051


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
### YOUR CODE HERE ###

cpu_cluster_name = "cpu-cluster"

# to check whether the compute cluster exists already or not
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Existing compute target found... Using it")

except ComputeTargetException:
    print("Creating new Compute Target... " + cpu_cluster_name)
    provisioning_compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Existing compute target found... Using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [11]:
from azureml.widgets import RunDetails
import shutil
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig, Experiment
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os


# Specify parameter sampler
ps = RandomParameterSampling({
    #"--penalty": choice('11', 'l2', 'elasticnet', 'none'),
    "--C" : choice([0.01, 0.5, 1.00]),
    "--max_iter" : choice([10, 100, 150, 200])
    #"--solver" : choice('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# os.makedirs(script_folder, exist_ok=True)
# shutil.copy('./train.py', script_folder)

# Create a SKLearn estimator for use with train.py

est = SKLearn(
    source_directory= os.path.join("./"),
    compute_target= cpu_cluster,
    entry_script= "train.py"
)


In [ ]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.

hyperdrive_config = HyperDriveConfig(
    estimator=est, 
    hyperparameter_sampling=ps, 
    policy=policy,
    primary_metric_name='accuracy', 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=20,
    max_concurrent_runs=4
)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
run = exp.submit(hyperdrive_config)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print('Best Run ID: ', best_run.id)
print('Best Run Accuracy: ', best_run_metrics['accuracy'])
print('Best Run Learning Rate: ', parameter_values[3])
print('Parameter Values: ', parameter_values)


In [ ]:
# registering the best model
best_run.get_file_names()

best_run.register_model(model_name="hyperdrive_best_model.pkl", model_path ="./outputs/model.joblib")

In [ ]:
best_run